# Dynamisches Labyrinth

Grundidee: 
A*-Algorithmus so anpassen, dass wenn durch eine fehlende Verbindung das erreichen des Ziels nicht möglich ist, sodass der beste Spielstein ermittelt und der Spieler darauf platziert wird. Danach wird die Spielfeldreihe verschoben und erneut nach dem Ziel gesucht. Der Beste Spielstein ist durch die Heuristik bewertet sowie der Eigenschaft, dass dieser einen Weg nach oben besitzt.
Konkreter in Kapitel 3.

## 1. CSV einlesen

In [1]:
import csv

# Pfad zur CSV-Datei
csv_datei_pfad = './data/Puzzle_3.csv'

# Liste zum Speichern der Daten
spielfeld = []

# CSV-Datei öffnen und Daten einlesen
with open(csv_datei_pfad, 'r') as csv_datei:
    csv_reader = csv.reader(csv_datei, delimiter=";")
    
    # Zeile für Zeile durch die CSV-Datei iterieren
    for zeile in csv_reader:
        spielfeld.append(zeile)

## 2. Spielsteine definieren

Erklärung: Es gibt 10 verschiedene Spielsteine die angeben, in welche Richtung der Spieler sich bewegen und nicht bewegen kann. Dabei werden die verschiedenen Spielsteine in einer
- Matrix von 0 - 9 dargestellt und
- jeder Spielstein erhält zudem eine 2x2 Matrix der die möglichen Richtungen angibt. 

Dabei gilt folgendes: [ left, top, right, bottom] wobei eine 1 für eine Verbindung und eine 0 für keine Verbindung steht zum nächsten Feld steht. Als Beipsiel, [0, 0, 1, 1] steht für [-, -, right, bottom], also ist dieser Spielstein eine Ecke die eine Verbindung von unten nach rechts besitzt.

In [2]:
# Kodierung der Spielsteine 
# [ left, top, right, bottom]
tile0 = [0, 0, 1, 1]
tile1 = [1, 0, 0, 1]
tile2 = [0, 1, 1, 0]
tile3 = [1, 1, 0, 0]
tile4 = [1, 0, 1, 1]
tile5 = [1, 1, 0, 1]
tile6 = [1, 1, 1, 0]
tile7 = [0, 1, 1, 1]
tile8 = [0, 1, 0, 1]
tile9 = [1, 0, 1, 0]

In [3]:
# Spielsteine und Kodierungen
tiles = [tile0, tile1, tile2, tile3, tile4, tile5, tile6, tile7, tile8, tile9]

## 2.2 Grid definieren
Die Klasse `Grid` verwaltet das Spielfeld für den A*-Algorithmus und die Spielsteinbewegungen. Sie initialisiert das Spielfeld, ermöglicht das Abrufen und Setzen von Knotenwerten sowie das Verschieben des Spielers und der Spielsteine. Die Methode `get_adjacent` ermittelt benachbarte Knoten, die durch begehbare Wege verbunden sind.

In [4]:
class Grid:
    '''
    Das Grid erleichtert das Arbeiten im A* Stern, verschieben der
    Spielsteine (Reihen) und des Spielers.
    '''
    def __init__(self, src, dst, matrix):
        '''
        Definiert Start und Ziel um Grid.
        Initialisiert alle Knoten und Knoten Werte

        Parameters: src, dst
        Returns: none
        '''
        self.src = src
        self.dst = dst
        self.off_tile = int(matrix[-1][0])
        self.rows = len(matrix)-1
        self.cols = len(matrix[0]) if self.rows > 0 else 0
        self.nodes = [(i, j) for i in range(self.rows) for j in range(self.cols)]
        self.grid_values = [[0 for _ in range(self.cols)] for _ in range(self.rows)]

        for i in range(self.rows):
            for j in range(self.cols):
                self.grid_values[i][j] = int(matrix[i][j])

    def get_value(self, row, col):
        '''
        Wert der zur Kodierung der Spielsteine verwendet wird 

        Parameters: row, col
        Returns: int
        '''
        if 0 <= row < self.rows and 0 <= col < self.cols:
            return self.grid_values[row][col]
        return None
    
    def set_value(self, row: int, col: int, new_value: int(0-9)):
        '''
        Spielstein Kodierungs Wert (0-9) um z.B. Spielsteine verschieben

        Parameters: row, col, new_value
        Returns: none
        '''
        self.grid_values[row][col] = int(new_value)

    def move_player_left(self, player_src):
        '''
        Verschiebt den Spieler nach links und gibt die neue Position zurück

        Parameters: player_src
        Returns: new_position
        '''
        row, col = player_src

        if col == 0:
            new_position = (row, self.cols - 1)
        else:
            new_position = (row, (col - 1) % self.cols)

        return new_position
    
    def move_row_left(self, row_index, new_tile):
        '''
        Verschiebt die gegebene Reihe (index) nach links, 
        setzt ein neuen Spielstein (new_tile) rechts ein und 
        gibt den entfernten linken Spielstein zurück

        Parameters: row_index, new_tile
        Returns: moved_out
        '''
        row = self.grid_values[row_index]
        moved_out = row[0]

        for i in range(1, self.cols):
            self.set_value(row_index, i - 1, row[i])
        self.set_value(row_index, self.cols - 1, new_tile)

        return moved_out
        
    def move_player_right(self, player_src):
        '''
        Verschiebt den Spieler nach rechts und gibt die neue Position zurück

        Parameters: player_src
        Returns: new_position
        '''
        row, col = player_src
        if col == self.cols - 1:
            new_position = (row, 0)
        else:
            new_position = (row, (col + 1) % self.cols)

        return new_position
    
    def move_row_right(self, row_index, new_tile):
        '''
        Verschiebt die gegebene Reihe (index) nach rechts, 
        setzt ein neuen Spielstein (new_tile) links ein und 
        gibt den entfernten rechten Spielstein zurück

        Parameters: row_index, new_tile
        Returns: moved_out
        '''
        row = self.grid_values[row_index]
        moved_out = row[-1]
        for i in range(self.cols - 1, 0, -1):
            self.set_value(row_index, i, row[i-1])
        self.set_value(row_index, 0, new_tile)

        return moved_out

    def get_nodes(self):
        '''
        Parameters: None
        Returns: nodes
        '''       
        return self.nodes
    
    def get_adjacent(self, node):   
        '''
        Ermittelt alle horizontal und vertikal liegenden Nachbarn eines Knotens.
        Gibt alle benachbarten Knoten zurück.

        Parameters: node
        Returns: adjacent_nodes
        '''     
        y, x = node
        adjacent_nodes = []

        # Horizontal Nachbarn
        for dx in [-1, 1]:
            new_x, new_y = x + dx, y
            # Wenn im Grid dann
            if 0 <= new_y < self.rows and 0 <= new_x < self.cols:
                # linker Nachbar
                if dx == -1:
                    possible_path_new_x = tiles[self.get_value(new_y, new_x)]
                    possible_path_x = tiles[self.get_value(y, x)]

                    # Sind die Nachbarn mit einem Weg verbunden? 
                    if possible_path_new_x[2] == 1 and possible_path_x[0] == 1:                   
                        adjacent_nodes.append((new_y, new_x))

                # rechter Nachbar
                if dx == 1:
                    possible_path_new_x = tiles[self.get_value(new_y, new_x)]
                    possible_path_x = tiles[self.get_value(y, x)]

                    # Sind die Nachbarn mit einem Weg verbunden?
                    if possible_path_new_x[0] == 1 and possible_path_x[2] == 1: 
                        adjacent_nodes.append((new_y, new_x))

        # Vertikal Nachbarn
        for dy in [-1, 1]:
            new_x, new_y = x, y + dy

            # Wenn im Grid dann
            if 0 <= new_y < self.rows and 0 <= new_x <= self.cols:
                # Oberer Nachbar 
                if dy == -1:
                    possible_path_new_y = tiles[self.get_value(new_y, new_x)]
                    possible_path_y = tiles[self.get_value(y, x)]

                    # Sind die Nachbarn mit einem Weg verbunden? 
                    if possible_path_new_y[3] == 1 and possible_path_y[1] == 1:
                        adjacent_nodes.append((new_y, new_x))

                # Unterer Nachbar
                if dy == 1:
                    possible_path_new_y = tiles[self.get_value(new_y, new_x)]
                    possible_path_y = tiles[self.get_value(y, x)]
                    # Sind die Nachbarn mit einem Weg verbunden? 
                    if possible_path_new_y[1] == 1 and possible_path_y[3] == 1:
                        adjacent_nodes.append((new_y, new_x))
                

        return adjacent_nodes

In [5]:
# Spielfeld Grid initialiseren aus "Spielfeld" (CSV Datei)
spielfeld_grid = Grid((4,0), (0,3), spielfeld)

## 3. A* anhand lösbarem Beispiel

Eine Kenngröße, die den A*-Algorithmus ausmacht, ist der heuristische Wert. In der Funktion des Algorithmus, in der die Wegkosten zum nächsten Knoten und genau dieser Wert der Heuristik, also die optimale Länge des Weges vom Standpunkt zum Ziel, addiert werden. Die verschiedenen Ergebniswerte werden miteinander verglichen, wobei der Weg mit dem niedrigsten Wert als nächstbesten anerkannt und verwendet wird.

Für die Berechnung des heuristischen Wertes haben wir uns für den "Manhatten"-Ansatz entschieden. Ein Schritt entspricht den Kosten 1. Anhand des Standortes, also dem momentanen Spielstein auf dem man steht, "läuft" man den direkten Weg zum Ziel und zählt die Schritte. Dabei werden jegliche Verbindungen der einzelnen Spielsteine ignoriert. Dieser optimale Weg lässt sich leicht aus den x und y werten berechnen, in dem man die Differenzen aus Start- (aktueller Standort) und Zielknotenkoordinaten nimmt. 
Um in unseren Fall den bestmöglichen Pfad innerhalb des Labyrinths zu finden, haben wir es als sinnvoll angesehen, Spielsteine mit einer Öffnung nach oben höher zu priorisieren, da dadurch bei Verschiebungen die Chance erhöht wird, früher einen Schritt näher in Richtung Ziel zu laufen.
Um diese Priorisierung zu erreichen, geben wir allen Spielsteinen die keine Öffnung nach oben haben eine "Strafe" von + Breite der Matrix minus eins (col -1) im heuristischen Wert.

In [6]:
def heuristic_cost_estimate(current, goal, grid):
    # Aktueller Standort und festgelegtes Ziel
    # Tupel in einzelne verwendbare Werte teilen.
    x1, y1 = current
    x2, y2 = goal

    # Spielsteinwert aus dem Grid lesen mit x1, y1
    grid_tile_number = grid.get_value(x1,y1)
    
    # Spielsteinmuster bestimmen durch Wert aus Grid
    grid_tile_value = tiles[grid_tile_number]
    
    # Wenn Tile mit Weg nach oben +0 (Priorisiertes Ziel)
    if grid_tile_value[1]:
        return abs(x2 - x1) + abs(y2 - y1)
    
    # Wenn Tile nicht mit Weg nach oben +n (n = Anzahl der Werte in einer Reihe)
    return abs(x2 - x1) + abs(y2 - y1) + grid.cols - 1

Der A*-Alg. hat folgende Merkmale: Es verwendet Knoten (nodes) die über Wegkosten (g_score) miteinander verbunden sind. Es gibt einen Start-Knoten (src) und einen Ziel-Knoten (dst), der erreicht werden soll. Der Funktionswert (f_score) des Alg. wird aus den Wegkosten und dem heuristischen Wert berechnet. Über Dictionarys speichern wir für die einzelnen Knoten die benötigtetn Werte ab, um diese im Alg. abrufen zu können. Der Alg. wird ebenfalls durch eine offene Liste und eine geschlossene Liste gekennzeichnet. In der offenen Liste werden immer zukünftige Knoten hineingeschrieben, die noch durch den Alg. geprüft werden müssen. In der geschlossenen werden die Knoten hineingeschrieben, die schon durch den Alg. gelaufen sind. In unserer Aufgabe spiegelt sich der Start-Knoten als unsere Spielfigur wider, da immer von dem momentanen Standort der Spielfigur der bestmögliche Weg gefunden werden soll.  

Der Alg. läuft solange durch, bis man das Ziel erreicht hat oder eine Abbruchbedingung erfüllt wurde. Im ersten Schritt wird überprüft, ob die Spielfigur das Spielfeld betreten hat, da bis dahin nur eine Reihe des Spielfeldes verschoben werden darf und nicht die Spielfigur selbst. Falls der Eintritt in das Spielfeld möglich ist, wird dies getan und der Startknoten als erster in die offene Liste geschoben. In der While-Schleife mit der Bedingung "open_set" wird jeder mögliche Nachbar verglichen und in die offene Liste geschrieben. Der Vergleich ist zwischen den heuristischen Werten. Es wird also geschaut, ob der mögliche Nachbar einen kürzeren Weg zum Ziel hat, als das momentane Feld. Da wir nur die heuristischen Werte vergleichen, setzen wir die Spielfigur an die Position, an der zum ersten Mal der niedrigste heuristishe Wert gefunden wurde. Denn je später der gleiche heuristische Werte gefunden wird, desto höher sind die Wegkosten und somit auch die Funktionskosten. Ist das Ziel durch nur laufen erreichbar, wird die while abgebrochen. Die Abbruchbedingung für den Alg. ist jedoch nicht nur am Ziel-Knoten anzukommen, sondern auch von diesem nach oben aus dem Spielfeld zu laufen. Somit wird noch überprüft, ob der Weg nach oben möglich ist. 
Ist das Ziel nach dem Laufen noch nicht erreicht bzw. die Öffnung nach oben nicht vorhanden, muss verschoben werden. Nachdem verschoben wurde, wird anhand des neuen Standortes der Alg. durchgelaufen.

In [7]:
def a_star(grid):
    # Initialisierung des Grids mit Start und Ziel
    nodes = grid.get_nodes()
    src = grid.src
    dst = grid.dst
    
    # Initialisierung von Dictionarys 
    g_score = {node: float("inf") for node in nodes}                # Dict. für die Wegkosten der einzelnen Knoten
    f_score = {node: float("inf") for node in nodes}                # Dict. für den insgesamten Funktionswert des Alg. der einzelnen Knoten
    
    # Wertzuweisung des Startpunktes
    g_score[src] = 0
    f_score[src] = heuristic_cost_estimate(src, dst, grid)
    
    # "best_node" spiegelt die Spielfigur wider und liegt auf dem "besten Knoten"
    best_node_cost = f_score[src]
    best_node = grid.src
    best_node_g_cost = 0
    
    # Initialisierung der offenen (mit Startpunkt) und geschlossenen Liste
    open_set = [best_node]
    closed_set = []

    # wichtig, da Spielfigur ausserhalb des Grids startet und erst "ins Spiel kommen" muss.
    # solange nicht im Spiel darf sich der Spieler nicht bewegen
    in_game = False
    schritte = 0

    # Alg. läuft durch bis Ende gefunden oder Abbruchbedingung !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    while True:
        # Überprüfung, ob Einstiegs-Spielstein nach unten geöffnet ist und ob Spielfigur schon im Spiel ist
        if tiles[grid.get_value(best_node[0],best_node[1])][3] != 1 and in_game == False:
            open_set.remove(best_node)

        # während Knoten in der offenen Liste sind, kann man laufen
        # ermittelt aus dem momentan möglichen Wegen den besten Weg und Standort
        # läuft diesen Weg bis zum gewünschten Standort
        while open_set:
            # aus der offenen Liste wird der Knoten mit den wenigsten Kosten ausgewählt und aus der offenen in die geschlossenen Liste verschoben
            u = min(open_set, key=f_score.get)
            open_set.remove(u)
            closed_set.append(u)
            
            # Falls der gewählte Knoten das gewünschte Ziel ist, beende das "laufen" 
            if u == dst:
                print('A* findet Ende')
                break
            else:
                # Vergleiche Werte mit jedem möglichen Nachbar und setze evtl. Standort neu
                for v in grid.get_adjacent(u):
                    # erhöhe Wegkosten für Nachbar um 1 (ein Schritt bis zum Nachbar)
                    new_g = g_score[u] - best_node_g_cost + 1
                    
                    # Überprüfe ob Knoten schon den Alg. durchgelaufen ist, falls nicht:
                    if v not in open_set and v not in closed_set:
                        #setze Nachbar in offene Liste und setze Wegkosten und Funktionswert
                        open_set.append(v)
                        g_score[v] = new_g
                        f_score[v] = new_g + heuristic_cost_estimate(v, dst, grid)
                        
                        # Überprüfe, ob momentan bester Standort-Knoten teuerer als der Knoten des Nachbarn
                        # wir wählen den erst Besten ("<") aufgrund der weiter benötigten Schritte (wenn kein Ende vorhanden)
                        # -> somit nur der Vergleich von heuristischem Wert nötig
                        if heuristic_cost_estimate(v, dst, grid) < best_node_cost:
                            schritte += 1
                            best_node = v
                            best_node_cost = heuristic_cost_estimate(v, dst, grid)
                            best_node_g_cost = new_g

                    # Falls ein kürzerer Weg zum gleichen Knoten gefunden wird:  
                    elif ((v in open_set) or (v in closed_set)) and (new_g < g_score[v]):
                        #setze Wegkosten und Funktionswert
                        g_score[v] = new_g
                        f_score[v] = new_g + heuristic_cost_estimate(v, dst, grid)
                        
                        # Von geschlossener Liste in offene Liste verschieben, da neue Weg-Möglichkeiten eröffnet wurden
                        if v in closed_set:
                            open_set.append(v)
                            closed_set.remove(v)

                        # Überprüfe, ob momentan bester Standort-Knoten teuerer als der Knoten des Nachbarn
                        # wir wählen den erst Besten ("<") aufgrund der weiter benötigten Schritte (wenn kein Ende vorhanden)
                        # -> somit nur der Vergleich von heuristischem Wert nötig
                        if heuristic_cost_estimate(v, dst, grid) < best_node_cost:
                            schritte += 1
                            best_node = v
                            best_node_cost = heuristic_cost_estimate(v, dst, grid)
                            best_node_g_cost = new_g
                    
        # Falls nach dem Laufen der Standort-Knoten das gewünschte Ziel ist UND der Spielstein nach oben aus dem Labyrinth geöffnet ist, beende den Alg.
        if best_node == dst and tiles[grid.get_value(dst[0],dst[1])][1] == 1: 
            schritte += 1
            print(tiles[grid.get_value(dst[0],dst[1])][1])
            print('Ziel zeigt nach oben.')
            break

        # Verschieben, da der Spieler ohne nicht das Ziel erreichen kann (vorherige Abbruchbedingung)
        if in_game:
            # Alle geraden Reihen nach rechts verschieben
            if (best_node[0] % 2 == 0):
                schritte += 1
                grid.off_tile = grid.move_row_right(best_node[0], grid.off_tile)
                best_node = grid.move_player_right(best_node) 
                
            # Alle ungeraden Reihen nach Links verschieben
            else: 
                schritte += 1
                grid.off_tile = grid.move_row_left(best_node[0], grid.off_tile)
                best_node = grid.move_player_left(best_node)
        
        # Falls Spielfigur noch nicht im Spiel, nur die Reihen und nicht die Spielfigur selbst verschieben
        else:
            if (best_node[0] % 2 == 0):
                schritte += 1
                grid.off_tile = grid.move_row_right(best_node[0], grid.off_tile)
            else: 
                schritte += 1
                grid.off_tile = grid.move_row_left(best_node[0], grid.off_tile)
        
        # Falls Spielfigur noch nicht im Spiel und Eintritt durch Öffnung nach unten möglich, gehe ins Spiel
        if tiles[grid.get_value(best_node[0],best_node[1])][3] == 1 and in_game == False:
            schritte += 1
            in_game = True
        
        # Initialisiert Neustart des Alg. vom neuen Standort aus
        open_set = [best_node]

        # Falls nach dem Verschieben der Standort-Knoten das gewünschte Ziel ist UND der Spielstein nach oben aus dem Labyrinth geöffnet ist, beende den Alg.
        if best_node == dst and tiles[grid.get_value(best_node[0],best_node[1])][1] == 1:
            print("Ziel zeigt nach oben nach dem Verschieben.")
            schritte += 1
            break  
    
    print("Benötigte Schritte ", schritte)


In [8]:
# Lösen des Puzzels durch den veränderten A*-Algorithmus
a_star(spielfeld_grid)

Ziel zeigt nach oben nach dem Verschieben.
Benötigte Schritte  12


## Fazit

Das lösen des Puzzels ist durch viele verschiedene Ansätze möglich. Grund dafür ist die vielzahl an verschiedenen Möglichkeiten einen optimalen Pfad vom Start zum Ziel.
In unserem Algorithmus haben wir die Suche nach dem allgemein perfekten Spielstein ausgeschlossen, da dieser einen "brute force" Ansatz verfolgt. Hierbei würden mehrere Wege verglichen werden und nicht nur, wie der A*-Algorithmus vorgibt, die benachbarten Knoten. Deshalb muss für die von uns entwickelte Lösung eine "lösbare" Matrix vorgegeben werden, die vorraussetzt, dass in anderen Reihen nicht nach einem passenden Spielstein zum weiterlaufen gesucht wird, sondern nur innerhalb der Reihe, in der der Spieler sich befindet.